In [1]:
from __future__ import print_function

from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout
from keras.layers import add, dot, concatenate
from keras.layers import LSTM
import numpy as np

from dataproc_utils import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
test_size = 0.2
random_state = 42
batch_size = 64
epochs = 25
body_size = 20
claim_size = 12
embedding_dim = 100
output_size = 4  # size of the output vector

In [3]:
# load data and labels
data = load_proc_data('train_bodies.txt', 'train_claims.txt', split_pars=False)
labels = [label for body, claim, label in data]
y = np.array(labels)

print('First input tuple (body, claim, stance):\n', data[0])


# train/validation/test split
train_data, val_data, train_labels, val_labels = train_test_split(data, y, test_size=.2, random_state=random_state)


# create a vocabulary dict from train data
word2freq = make_word_freq_V(train_data)
word2index = word2idx(word2freq)

vocab_size = len(word2index) + 1
print('Vocab size:', vocab_size, 'unique words in the train set')

# vectorize input words (turn each word into its index from the word2index dict)
# for new words in test set that don't appear in train set, use index of <unknown>
train_body, train_claim = word_vectorizer(train_data, word2index, max_body_len=body_size, max_claim_len=claim_size)
val_body, val_claim = word_vectorizer(val_data, word2index, max_body_len=body_size, max_claim_len=claim_size)


# perform random under/over sampling to prevent class imbalance
train_body, train_claim, train_labels = random_sampler(train_body, train_claim, train_labels, type='over')


print("Training set shape: bodies", train_body.shape)
print("Training set shape: claims", train_claim.shape)
print("Training set shape: labels", train_labels.shape)

# data size params
n_train = train_body.shape[0]
n_val = val_body.shape[0]

print("Training Size", n_train)
print("Validation Size", n_val)


# initialize placeholders
input_body = Input((body_size,))
input_claim = Input((claim_size,))

First input tuple (body, claim, stance):
 (['seth', 'rogen', 'is', 'being', 'eyed', 'to', 'play', 'apple', 'co-founder', 'steve', 'wozniak', 'in', 'sonys', 'steve', 'jobs', 'biopic', 'danny', 'boyle', 'is', 'directing', 'the', 'untitled', 'film', 'based', 'on', 'walter', 'isaacson', "'s", 'book', 'and', 'adapted', 'by', 'aaron', 'sorkin', 'which', 'is', 'one', 'of', 'the', 'most', 'anticipated', 'biopics', 'in', 'recent', 'years', 'negotiations', 'have', 'not', 'yet', 'begun', 'and', 'its', 'not', 'even', 'clear', 'if', 'rogen', 'has', 'an', 'official', 'offer', 'but', 'the', 'producers', 'scott', 'rudin', 'guymon', 'casady', 'and', 'mark', 'gordon', 'have', 'set', 'their', 'sights', 'on', 'the', 'talent', 'and', 'are', 'in', 'talks', 'of', 'course', 'this', 'may', 'all', 'be', 'for', 'naught', 'as', 'christian', 'bale', 'the', 'actor', 'who', 'is', 'to', 'play', 'jobs', 'is', 'still', 'in', 'the', 'midst', 'of', 'closing', 'his', 'deal', 'sources', 'say', 'that', 'dealmaking', 'proces

In [14]:
# LSTM !!!!
# encoders
# embed the input sequence into a sequence of vectors
body_encoder_m = Sequential()
body_encoder_m.add(Embedding(input_dim=vocab_size,
                              output_dim=embedding_dim))
body_encoder_m.add(Dropout(0.3))  # output: (samples, body_size, embedding_dim)

In [5]:
# CNN!!!!!
# embed the input into a sequence of vectors of size claim_size
body_encoder_c = Sequential()
body_encoder_c.add(Embedding(input_dim=vocab_size,
                             output_dim=claim_size))
body_encoder_c.add(Dropout(0.3))  # output: (samples, body_size, claim_size)

In [6]:
# embed the claim into a sequence of vectors
claim_encoder = Sequential()
claim_encoder.add(Embedding(input_dim=vocab_size,
                            output_dim=embedding_dim,
                            input_length=claim_size))
claim_encoder.add(Dropout(0.3))  # output: (samples, claim_size, embedding_dim)

In [7]:
# encode article bodies and claims (which are indices) as sequences of dense vectors
body_encoded_m = body_encoder_m(input_body)
body_encoded_c = body_encoder_c(input_body)
claim_encoded = claim_encoder(input_claim)

In [8]:
# compute a 'match' between the body vector sequence and the claim vector sequence
match = dot([body_encoded_m, claim_encoded], axes=(2, 2))
match = Activation('softmax')(match)  # shape: (samples, body_size, claim_size)

In [9]:
# add the match matrix with the second input vector sequence
response = add([match, body_encoded_c])  # (samples, body_size, claim_size)
response = Permute((2, 1))(response)  # (samples, claim_size, body_size)

In [10]:
# concatenate the match matrix with the claim vector sequence
stance = concatenate([response, claim_encoded])

In [11]:
# the original paper uses a matrix multiplication for this reduction step.
# we choose to use a RNN instead.
stance = LSTM(32)(stance)  # shape: (samples, 32)

In [12]:
# one regularization layer -- more would probably be needed.
stance = Dropout(0.3)(stance)
stance = Dense(output_size)(stance)  # (samples, output_size)
# we output a probability distribution over the four stances
preds = Activation('softmax')(stance)

In [13]:
# build the model
model = Model([input_body, input_claim], preds)
model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


# train
model.fit([train_body, train_claim], train_labels,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=([val_body, val_claim], val_labels))

# print model summary
print(model.summary())

Train on 116848 samples, validate on 9995 samples
Epoch 1/25
116848/116848 [==============================] - 50s 425us/step - loss: 0.7885 - acc: 0.6523 - val_loss: 1.0318 - val_acc: 0.4723
Epoch 2/25
116848/116848 [==============================] - 46s 392us/step - loss: 0.5505 - acc: 0.7768 - val_loss: 0.7219 - val_acc: 0.6737
Epoch 3/25
116848/116848 [==============================] - 46s 393us/step - loss: 0.4492 - acc: 0.8250 - val_loss: 0.6841 - val_acc: 0.6996
Epoch 4/25
116848/116848 [==============================] - 46s 392us/step - loss: 0.3910 - acc: 0.8507 - val_loss: 0.7181 - val_acc: 0.6965
Epoch 5/25
116848/116848 [==============================] - 46s 392us/step - loss: 0.3499 - acc: 0.8702 - val_loss: 0.6208 - val_acc: 0.7552
Epoch 6/25
116848/116848 [==============================] - 46s 393us/step - loss: 0.3152 - acc: 0.8850 - val_loss: 0.6199 - val_acc: 0.7593
Epoch 7/25
116848/116848 [==============================] - 46s 392us/step - loss: 0.2830 - acc: 0.8976 